# Stock Market Movement Prediction
### Using Machine Learning to Forecast Next-Day Stock Price Movements
This project tackles a binary classification problem in financial markets -
Anal
predicting whether individual US stocks will move up or down the following day.
The goal is to develop a model that can assist in making data-driven investment decisions.
The model uses 20 days of historical price returns and trading volumes, along with
categorical stock metadata like industry and sector classifications, to identify
predictive patterns in market behavior.
The public benchmark accuracy of 51.31% was achieved using a Random Forest model that considered
the previous 5 days of data along with the average sector returns from the prior day.

#### Agenda
1. **Data Preprocessing**
   - Loading training and test datasets
   - Handling missing values and target encoding
   - Feature engineering (technical indicators of different types)
2. **Model Implementation and Evaluation**
   - Decision Tree Classifier
      - Baseline model (accuracy: 0.510)
      - Tuned model with hyperparameter optimization (accuracy: 0.5325)
   - XGBoost Classifier
      - Baseline model (accuracy: 0.53)
      - Tuned model with hyperparameter optimization (accuracy: 0.8775)
   - Neural Network
      - Accuracy: 0.5144
3. **Model Comparison**
   - Cross-validation results
   - Feature importance analysis
   - ROC curves and confusion matrices


## Data description

3 datasets are provided as csv files, split between training inputs and outputs, and test inputs.

Input datasets comprise 47 columns: the first ID column contains unique row identifiers while the other 46 descriptive features correspond to:

* **DATE**: an index of the date (the dates are randomized and anonymized so there is no continuity or link between any dates),
* **STOCK**: an index of the stock,
* **INDUSTRY**: an index of the stock industry domain (e.g., aeronautic, IT, oil company),
* **INDUSTRY_GROUP**: an index of the group industry,
* **SUB_INDUSTRY**: a lower level index of the industry,
* **SECTOR**: an index of the work sector,
* **RET_1 to RET_20**: the historical residual returns among the last 20 days (i.e., RET_1 is the return of the previous day and so on),
* **VOLUME_1 to VOLUME_20**: the historical relative volume traded among the last 20 days (i.e., VOLUME_1 is the relative volume of the previous day and so on),

Output datasets are only composed of 2 columns:

* **ID**: the unique row identifier (corresponding to the input identifiers)
and the binary target:
* **RET**: the sign of the residual stock return at time $t$

------------------------------------------------------------------------------------------------
The one-day return of a stock :
$$R^t = \frac{P_j^t}{P_j^{t-1}} - 1$$

The volume is the ratio of the stock volume to the median volume of the past 20 days.
The relative volumes are computed using the median of the past 20 days' volumes.
If any day within this 20-day window has a missing volume value, it will cause NaN values in the calculation for subsequent days.
For example, if there is a missing value on day $D$, then the relative volumes for days $D$ to $D+19$ will be affected.

The relative volume $\tilde{V}^t_j$ at time $t$ of a stock $j$ is calculated as:
$$
\tilde{V}^t_j = \frac{V^t_j}{\text{median}( \{ V^{t-1}_j, \ldots, V^{t-20}_j \} )}
$$

The adjusted relative volume $V^t_j$ is then given by:
$$
V^t_j = \tilde{V}^t_j - \frac{1}{n} \sum_{i=1}^{n} \tilde{V}^t_i
$$
------------------------------------------------------------------------------------------------
Guidelines from the organizers:
The solution files submitted by participants shall follow this output dataset format (i.e contain only two columns, ID and RET, where the ID values correspond to the input test data).
An example submission file containing random predictions is provided.

**418595 observations (i.e. lines) are available for the training datasets while 198429 observations are used for the test datasets.**


## Implementation Steps

This notebook implements the following steps:

1. **Data Loading and Preprocessing**
   - Load training and test datasets
   - Handle missing values and data cleaning
   - Calculate technical indicators using TA-Lib
   - Filter out infinity values and remove duplicated columns
   - Split data into training and test sets (75%/25% split)

2. **Feature Engineering**
   - Calculate technical indicators like RSI, OBV, EMA etc.
   - Save indicators to pickle files for reuse
   - Drop unnecessary ID and categorical columns
   - Remove redundant technical indicators

3. **Model Development and Tuning**
   - Decision Tree Classifier
      - Baseline model (accuracy: 0.510)
      - Tuned model with hyperparameters (accuracy: 0.533)
   - Gradient Boosting
      - Stepwise tuning of n_estimators, tree params, leaf params
      - Best model achieves significant improvement
   - Neural Network
      - Simple feed-forward architecture
      - Training with BCE loss and Adam optimizer

4. **Model Comparison and Analysis**
   - Compare accuracy across all models
   - Analyze feature importance
   - Key findings on model performance and technical indicators
   - Discussion of overfitting and benchmark results

### Importing libraries

In [14]:
import sys
from pathlib import Path

try:
    # vscode
    path = Path(__file__).parent.parent
    path = path / "src"
except NameError:
    # jupyter notebook
    path = Path().absolute().parent
sys.path.append(str(path))

import kedro.ipython
from kedro.ipython import get_ipython

kedro.ipython.load_ipython_extension(get_ipython())

[01/18/25 14:06:15] INFO     Registered line magic '%reload_kedro'                                   ]8;id=536797;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=612902;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/ipython/__init__.py#61\61]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=648467;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=466403;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Resolved project path as:                                              ]8;id=941899;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=519152;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             /Users/krzysztofwojdalski/github_projects/ml-in-finance-i-project.                    
                             To set a different path, run '%reload_kedro <project_root>'                           

/Users/krzysztofwojdalski/github_projects/ml-in-finance-i-project/src/ml_in_finance_i_project


[01/18/25 14:06:16] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=751668;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=597077;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

                    INFO     Kedro project ml_in_finance_i_project                                  ]8;id=362458;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=962287;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=810621;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=612964;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=706540;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=85469;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/ipython/__init__.py#151\151]8;;\

In [15]:
import logging as log

from IPython.display import Markdown as md

from src.ml_in_finance_i_project.utils import get_node_idx, get_node_outputs

In [16]:
# Load the datasets
x_train_raw = catalog.load("x_train_raw")
y_train_raw = catalog.load("y_train_raw")
x_test_raw = catalog.load("x_test_raw")

[01/18/25 14:06:21] INFO     Loading data from x_train_raw (CSVDataset)...                      ]8;id=75700;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=496609;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/io/data_catalog.py#389\389]8;;\

[01/18/25 14:06:22] INFO     Loading data from y_train_raw (CSVDataset)...                      ]8;id=228693;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=662274;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from x_test_raw (CSVDataset)...                       ]8;id=659266;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=154827;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/io/data_catalog.py#389\389]8;;\

### Checking and configuring environment

In [17]:
## Google Colab used to speed up the computation in xgboost model
## warning: this function must be run before importing libraries
# if running in Google Colab
def setup_colab_environment():
    """
    Set up Google Colab environment by mounting drive and creating symlinks.
    Returns True if running in Colab, False otherwise.
    """
    try:
        import os

        from google.colab import drive

        drive.mount("/content/drive")
        req_symlinks = [
            ("data", "ml_in_finance_i_project/data"),
            ("src", "ml_in_finance_i_project/src"),
        ]
        # Create symlinks if they don't exist
        for dest, src in req_symlinks:
            if not os.path.exists(dest):
                os.symlink(f"/content/drive/Othercomputers/My Mac/{src}", dest)
        return True

    except ImportError:
        return False

#### Run pipeline node definition. This one must be evaluated within the notebook

In [18]:
def run_pipeline_node(pipeline_name: str, node_name: str, inputs: dict):
    """Run a specific node from a pipeline.

    Args:
        pipeline_name: Name of the pipeline
        node_name: Name of the node to run
        inputs: Dictionary of input parameters for the node

    Returns:
        Output from running the node
    """
    node_idx = get_node_idx(pipelines[pipeline_name], node_name)
    return pipelines[pipeline_name].nodes[node_idx].run(inputs)

##### Setup environment

In [19]:
IN_COLAB = setup_colab_environment()

# Configure logging to stdout
log.basicConfig(
    level=log.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[log.StreamHandler()],
)
conf_params = context.config_loader.get("parameters")
target = conf_params["model_options"]["target"]
kfold = conf_params["model_options"]["kfold"]

## Loading data

#### Preprocessing data
* Dropping rows with NAs
* RET is encoded from bool to binary
* [Optional] Loading only a fraction of the dataset to speed up the computation
* [Optional] Trimming the dataset to the first n days (for instance, `n_days=5` mean we will only load `RET_1` to `RET_5`)

In [20]:
# Set data directory based on environment
# Run data processing pipeline node
out = run_pipeline_node(
    "data_processing",
    "load_and_preprocess_data_node",
    {
        "x_train_raw": x_train_raw,
        "y_train_raw": y_train_raw,
        "x_test_raw": x_test_raw,
        "params:remove_id_cols": conf_params["remove_id_cols"],
        "params:n_days": conf_params["n_days"],
        "params:sample_n": conf_params["sample_n"],
    },
)

[01/18/25 13:19:09] INFO     Running node: load_and_preprocess_data_node:                               ]8;id=678628;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py\node.py]8;;\:]8;id=759724;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py#367\367]8;;\
                             load_and_preprocess_data([x_train_raw;y_train_raw;x_test_raw;params:remove            
                             _id_cols;params:n_days;params:sample_n]) -> [train_df;test_df]                        

#### Problem visualization

* Plot returns and volume
* Red bar is the target variable (RET) we are trying to predict

In [33]:
# Plot returns and volume
run_pipeline_node(
    "reporting",
    "plot_returns_volume_node",
    {
        "train_df": out["train_df"],
        "params:example_row_id": 2,
    },
)["returns_volume_plot"]

[01/18/25 13:27:22] INFO     Running node: plot_returns_volume_node:                                    ]8;id=969223;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py\node.py]8;;\:]8;id=971565;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py#367\367]8;;\
                             plot_ret_and_vol([train_df;params:example_row_id]) ->                                 
                             [returns_volume_plot]                                                                 

#### Glimpse of the data

* As we can see, the raw data is not clean (NaN values are visible)
* We will need to clean it up in the next step

In [22]:
out["test_df"].head()

,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,VOLUME_2,...,RET_1_SUB_INDUSTRY_median,RET_1_SUB_INDUSTRY_std,RET_2_SUB_INDUSTRY_median,RET_2_SUB_INDUSTRY_std,RET_3_SUB_INDUSTRY_median,RET_3_SUB_INDUSTRY_std,RET_4_SUB_INDUSTRY_median,RET_4_SUB_INDUSTRY_std,RET_5_SUB_INDUSTRY_median,RET_5_SUB_INDUSTRY_std
ID,,,,,,,,,,,,,,,,,,,,,
602753,215,712,53,19,7,126,0.020782,-0.432386,0.016149,-0.091659,...,0.017591,0.005053,0.016149,0.006815,0.001227,0.006519,-0.007307,0.005980,-0.015588,0.021054
540249,152,3677,27,8,4,66,-0.004720,-0.552429,0.009311,-0.667230,...,-0.005600,0.007644,0.009311,0.003188,-0.000182,0.002441,0.003651,0.013475,-0.013314,0.018190
525141,133,2571,34,11,5,85,-0.010844,0.105083,-0.030768,-0.023836,...,-0.009700,0.001619,-0.022414,0.011815,-0.000185,0.004249,-0.017408,0.000480,-0.018167,0.000826
567011,176,4525,41,14,6,101,-0.000775,-0.543201,0.002331,-0.338408,...,-0.000775,NaN,0.002331,NaN,-0.000776,NaN,-0.001550,NaN,-0.000775,NaN
508581,114,4180,66,24,9,162,-0.035664,0.108727,-0.026389,1.278625,...,-0.035664,NaN,-0.026389,NaN,0.015515,NaN,-0.001409,NaN,-0.011142,NaN


#### Info about the dataset

In [23]:
print("Training Dataset Info:")
out["train_df"].info()
print("\nTest Dataset Info:")
out["test_df"].info()

Training Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 43340 to 177228
Data columns (total 87 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   DATE                         10000 non-null  int64  
 1   STOCK                        10000 non-null  int64  
 2   INDUSTRY                     10000 non-null  int64  
 3   INDUSTRY_GROUP               10000 non-null  int64  
 4   SECTOR                       10000 non-null  int64  
 5   SUB_INDUSTRY                 10000 non-null  int64  
 6   RET_1                        10000 non-null  float64
 7   VOLUME_1                     10000 non-null  float64
 8   RET_2                        10000 non-null  float64
 9   VOLUME_2                     10000 non-null  float64
 10  RET_3                        10000 non-null  float64
 11  VOLUME_3                     10000 non-null  float64
 12  RET_4                        10000 non-null  float6

#### Plot nan percentages across categorical var


Possible reasons for missing values:
1. Market closures (market data might be missing for weekends and public holidays) - some dates clearly show a very high
percentage of missing values
2. Data collection issues (for instance, market data might come from different US venues, e.g. NYSE, NASDAQ, CBOE, etc.)
3. Randomization and anonymization of dates
4. The way relative volumes are calculated (one day missing causes missing values for the next 19 days) - could have something to do with calculating volumes on weekends / public holidays
5. Done on purpose by the organizers to make the problem more challenging
6. Some stocks might be delisted or suspended from trading (reference data problem) - some stocks in fact have up to 100% missing values
7. Some stocks might be barely trading (either due to low volume or in a non-continuous manner)

In [24]:
run_pipeline_node(
    "reporting",
    "plot_nan_percentages_node",
    {"train_df": out["train_df"]},
)["nan_percentages_plot"]

[01/18/25 14:10:16] INFO     Running node: plot_nan_percentages_node: plot_nan_percentages([train_df])  ]8;id=267390;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py\node.py]8;;\:]8;id=602620;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py#367\367]8;;\
                             -> [nan_percentages_plot]                                                             

#### Check Class Imbalance

Classes seem to be balanced almost perfectly. This is expected, as the target variable is the sign of the return.
Intuitively, it is expected that the sign of the return is more likely to be positive (by a small margin) than negative
unless data comes from a bear market period.

In [25]:
md(
    f"Class imbalance: {out['train_df']['RET'].value_counts(normalize=True)[0] * 100:.2f}%"
    + f" {out['train_df']['RET'].value_counts(normalize=True)[1] * 100:.2f}%"
)

Class imbalance: 50.02% 49.98%

#### Plot correlation matrix

Findings:
* Most stock returns are nearly not correlated with each other (this is expected).
Otherwise, someone could make a lot of money
by exploiting this non-subtle pattern.
    * Eventually, excess alpha would converge to 0
* Among stock returns the strongest correlation is within stock returns adjacent to each other (e.g. $RET_1$ and $RET_2$)
    * This is expected as the magnitude return of a stock is likely to be correlated with the magnitude stock return of the previous day
* Volumes are highly correlated (this is kind of expected) due to the way $VOLUME_i$ variables are calculated.
Moreover, Volatility and Volumes tend to cluster. Hence, correlation is positive.
* There is a strong positive correlation between the volume of the previous day and the return of the following day

In [13]:
out_corr = run_pipeline_node(
    "reporting",
    "plot_correlation_matrix_node",
    {"train_df": out["train_df"]},
)
out_corr["correlation_matrix_plot"]

                    INFO     Running node: plot_correlation_matrix_node:                                ]8;id=153549;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py\node.py]8;;\:]8;id=189289;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py#367\367]8;;\
                             plot_correlation_matrix([train_df]) -> [correlation_matrix_plot]                      

25/01/18 13:18:26 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Feature engineering - cont'd

* This comes from organizers' notebook, it's an extended version of variables they used
* Feature engineering
* Calculate statistical features

In [26]:


import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    out2 = run_pipeline_node(
        "data_processing",
        "calculate_statistical_features_node",
        {
            "train_df": out["train_df"],
            "test_df": out["test_df"],
        },
    )

[01/18/25 14:18:33] INFO     Running node: calculate_statistical_features_node:                         ]8;id=508723;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py\node.py]8;;\:]8;id=948230;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py#367\367]8;;\
                             calculate_statistical_features([train_df;test_df]) ->                                 
                             [train_df_statistical_features;test_df_statistical_features;statistical_fe            
                             atures]                                                                               

### Feature Engineering - Technical Indicators using TA-Lib

In this part, we calculate the technical indicators for the train and test data.
We save the results in pickle files to avoid recalculating them every time.
The following functions inside the function are used:
- talib.OBV, {"data_type": "both"}),
- talib.RSI, {"data_type": "ret"}),
- talib.MOM, {"timeperiod": 5, "data_type": "ret"}),
- talib.ROCR, {"timeperiod": 5, "data_type": "ret"}),
- talib.CMO, {"timeperiod": 14, "data_type": "ret"}),
- talib.EMA, {"timeperiod": 5, "data_type": "ret"}),
- talib.SMA, {"timeperiod": 5, "data_type": "ret"}),
- talib.WMA, {"timeperiod": 5, "data_type": "ret"}),
- talib.MIDPOINT, {"timeperiod": 10, "data_type": "ret"}),

----

This part has optional calculation of technical indicators as it typically takes a long time to compute (unless we use a small subset of the data)

In [27]:
calculate = True

if calculate:
    out3 = run_pipeline_node(
        "data_processing",
        "calculate_technical_indicators_node",
        {
            "train_df_statistical_features": out2["train_df_statistical_features"],
            "test_df_statistical_features": out2["test_df_statistical_features"],
            "params:features_ret_vol": conf_params["features_ret_vol"],
        },
    )
else:
    out5 = get_node_outputs(
        pipelines["data_processing"].nodes[
            get_node_idx(
                pipelines["data_processing"], "calculate_technical_indicators_node"
            )
        ],
        catalog,
    )

[01/18/25 14:18:34] INFO     Running node: calculate_technical_indicators_node:                         ]8;id=910486;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py\node.py]8;;\:]8;id=556183;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py#367\367]8;;\
                             calculate_technical_indicators([train_df_statistical_features;test_df_stat            
                             istical_features;params:features_ret_vol]) ->                                         
                             [train_ta_indicators;test_ta_indicators]                                              

### Columns to drop

* ID cols can be used after all features are calculated as portion of them were used.
* ID cols could bring in some predictive power, but we don't want to use them in this case
* We drop the following columns: `ID`, `STOCK`, `DATE`, `INDUSTRY`, `INDUSTRY_GROUP`, `SECTOR`, `SUB_INDUSTRY`

In [28]:

out4 = run_pipeline_node(
    "data_processing",
    "drop_id_cols_node",
    {
        "train_ta_indicators": out3["train_ta_indicators"],
        "test_ta_indicators": out3["test_ta_indicators"],
    },
)

[01/18/25 14:19:03] INFO     Running node: drop_id_cols_node:                                           ]8;id=201413;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py\node.py]8;;\:]8;id=731591;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py#367\367]8;;\
                             drop_id_cols([train_ta_indicators;test_ta_indicators]) ->                             
                             [train_ta_indicators_dropped;test_ta_indicators_dropped]                              

#### Drop obsolete technical indicators

* Assumption is that probably some technical indicators are not useful for the prediction and they can overfit the model
* For instance SMA(10), SMA(11) etc. dont give any information in the context of RET.
* It's an arbitrary choice, but we want to keep the number of features low

In [29]:
out5 = run_pipeline_node(
    "data_processing",
    "drop_obsolete_technical_indicators_node",
    {
        "train_ta_indicators_dropped": out4["train_ta_indicators_dropped"],
        "test_ta_indicators_dropped": out4["test_ta_indicators_dropped"],
        "params:target": conf_params["model_options"]["target"],
    },
)

                    INFO     Running node: drop_obsolete_technical_indicators_node:                     ]8;id=982098;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py\node.py]8;;\:]8;id=740229;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py#367\367]8;;\
                             drop_obsolete_technical_indicators([train_ta_indicators_dropped;test_ta_in            
                             dicators_dropped;params:target]) ->                                                   
                             [train_df_technical_indicators;test_df_technical_indicators;features]                 

#### Filter infinity values

* Filtering out infinity values as they can cause problems with model calculation (for instance, neural networks can't handle them)

In [30]:
out6 = run_pipeline_node(
    "data_processing",
    "filter_infinity_values_node",
    {
        "train_df_technical_indicators": out5["train_df_technical_indicators"],
        "test_df_technical_indicators": out5["test_df_technical_indicators"],
        "params:target": conf_params["model_options"]["target"],
    },
)

                    INFO     Running node: filter_infinity_values_node:                                 ]8;id=874436;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py\node.py]8;;\:]8;id=981163;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py#367\367]8;;\
                             filter_infinity_values([train_df_technical_indicators;test_df_technical_in            
                             dicators;params:target]) ->                                                           
                             [train_df_filtered;test_df_filtered;features_filtered]                                

                    WARNING  Train and test datasets have different available features                 ]8;id=366867;file:///Users/krzysztofwojdalski/github_projects/ml-in-finance-i-project/src/ml_in_finance_i_project/pipelines/data_processing/nodes.py\nodes.py]8;;\:]8;id=866602;file:///Users/krzysztofwojdalski/github_projects/ml-in-finance-i-project/src/ml_in_finance_i_project/pipelines/data_processing/nodes.py#256\256]8;;\

#### Remove duplicated columns and handle NaN values

* Before data is passed to the model, we need to make sure that no duplicated columns and NaN values are present
* NaN has been previously handled but this was before calculation of technical indicators. For this reason, we need to handle NaN values again

In [31]:
# Remove duplicated columns and handle NaN values
out7 = run_pipeline_node(
    "data_processing",
    "remove_duplicates_and_nans_node",
    {
        "train_df_filtered": out6["train_df_filtered"],
        "test_df_filtered": out6["test_df_filtered"],
    },
)

                    INFO     Running node: remove_duplicates_and_nans_node:                             ]8;id=344092;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py\node.py]8;;\:]8;id=918803;file:///Users/krzysztofwojdalski/.pyenv/versions/3.12.5/lib/python3.12/site-packages/kedro/pipeline/node.py#367\367]8;;\
                             remove_duplicates_and_nans([train_df_filtered;test_df_filtered]) ->                   
                             [train_df_clean;test_df_clean]                                                        